<a href="https://colab.research.google.com/github/Leah-Wu/SI650/blob/main/SI650_final_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get raw data and set up

In [1]:
!wget -nc http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz
!wget -nc http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Electronics.json.gz
!gzip -d reviews_Electronics_5.json.gz
!gzip -d meta_Electronics.json.gz

--2020-12-12 17:08:55--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 495854086 (473M) [application/x-gzip]
Saving to: ‘reviews_Electronics_5.json.gz’

reviews_Electronics 100%[===================>] 472.88M  9.00MB/s    in 60s     

2020-12-12 17:09:55 (7.83 MB/s) - ‘reviews_Electronics_5.json.gz’ saved [495854086/495854086]

--2020-12-12 17:09:55--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Electronics.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 186594679 (178M) [application/x-gzip]
Saving to: ‘meta_Electronics.json.gz’

meta_Electronics.js 100%[===

In [1]:
import pandas as pd
import json
from gensim import corpora, models, similarities
from nltk.corpus import stopwords
from datetime import timedelta
import re
import datetime
import nltk
import pprint
import nltk.data

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
import json
import sqlite3
import ast

from gensim import corpora, models, similarities
from nltk.corpus import stopwords
from datetime import timedelta
import re
import datetime
import nltk
import pprint
import nltk.data

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import json
import sqlite3
import ast

import string
import nltk
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from textblob import TextBlob

REMOVE_PUNCTUATION_TABLE = str.maketrans({x: None for x in string.punctuation})
TOKENIZER = TreebankWordTokenizer()
STEMMER = PorterStemmer()

In [3]:
pip install textblob

In [8]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Load data from dataset

In [3]:
reviews_Musical_Instruments_5 = pd.read_json('reviews_Electronics_5.json',lines=True)
meta_list = []
with open('meta_Electronics.json') as f:
  for line in f.readlines():
    line = json.dumps(ast.literal_eval(line))
    d = json.loads(line)
    if 'title' not in d:
      continue
    else:
      title = d['title']
      product_id = d['asin']
      img_url = d['imUrl']
  
      meta = []
      meta.append(product_id)
      meta.append(title)
      meta.append(img_url)
      meta_list.append(meta)
column_names = ['asin','title','image_url']
meta = pd.DataFrame(meta_list, columns=column_names)
entire_merged_df = reviews_Musical_Instruments_5[['asin','overall','reviewerID','reviewText']].merge(meta[['asin', 'title','image_url']], on='asin', how='inner')
entire_merged_df['reviewText'] = entire_merged_df['reviewText'].str.lower()
asin_review_df = entire_merged_df[['asin','reviewText']]
product_dict = dict(zip(entire_merged_df.asin, entire_merged_df.title))
product_dict =  {k: v.lower() for k, v in product_dict.items()}
review_list=list(entire_merged_df['reviewText'])

main function here:

In [4]:
entire_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1643686 entries, 0 to 1643685
Data columns (total 6 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   asin        1643686 non-null  object
 1   overall     1643686 non-null  int64 
 2   reviewerID  1643686 non-null  object
 3   reviewText  1643686 non-null  object
 4   title       1643686 non-null  object
 5   image_url   1643686 non-null  object
dtypes: int64(1), object(5)
memory usage: 87.8+ MB


# Main functions

Get queried dataset and the list of reviewText lists.

In [5]:
def get_review_list(query,entire_merged_df):
  queried_df = entire_merged_df[entire_merged_df['title'].str.contains(query)]
  review_list = list(queried_df['reviewText'])
  return review_list

In [6]:
def query_dataframe(query,entire_merged_df):
  queried_df = entire_merged_df[entire_merged_df['title'].str.contains(query)]
  #print(queried_df)
  review_list = list(queried_df['reviewText'])
  #print(review_list)
  review_list_of_list = [[x] for x in review_list]

  return review_list_of_list

Compute similarity and data preprocessing.


In [7]:
def similarity(query, input):

	valid_words = getdata([query])
  
	texts = []
	for i in input:
		texts.append(getdata(i))

	dictionary = corpora.Dictionary(texts)

	new_xs = dictionary.doc2bow(valid_words)
	corpus = [dictionary.doc2bow(text) for text in texts]
	  
	tfidf = models.TfidfModel(corpus)
	featurenum = len(dictionary.token2id.keys())
	index = similarities.SparseMatrixSimilarity(tfidf[corpus],
	                                          num_features=featurenum)
	sim = index[tfidf[new_xs]]
	sim_list = sim.tolist()
	#print(len(sim_list))
  # print(len(sim_list))
	return sim_list

In [8]:
def getdata(texts):
	textTokenized = [[word.lower() for word in word_tokenize(text)] for
                    text in texts]

	englishStopwords = stopwords.words('english')

	textFilterStopwords = [
      [word for word in text if not word in englishStopwords] for text in
      textTokenized]

	englishPunctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']',
                          '&', '!', '*', '@', '#', '$', '%', '<', '>',
                          '=', '{', '}', '+', '"', '-', '/', "'", '"']
	text_no_pun = [
      [word for word in text if word not in englishPunctuations]
      for text in textFilterStopwords]

	st = PorterStemmer()
	textStemed = [[st.stem(word) for word in text] for text in text_no_pun]
	flattened = [val for sublist in textStemed for val in sublist]
	return flattened

Get the result similarity dataframe without feedback.

In [9]:
def get_result_similarity_wtfeedback(sim_list, review_list_of_list):
  result_index_list = [i for i, e in enumerate(sim_list) if e != 0]
  result_sim_dict = {}
  for index in result_index_list:
    result_sim_dict[review_list_of_list[index][0]] = sim_list[index]
  result_review_list = []
  for index in result_index_list:
    result_review_list.append(review_list_of_list[index])
  result_sim_df = pd.DataFrame(list(result_sim_dict.items()),columns = ['reviewText','similarity']) 

  result_df = pd.DataFrame()
  for review in result_review_list:
    result_df = result_df.append(asin_review_df.loc[asin_review_df['reviewText'] == review[0]], ignore_index=True)
  
  pol = lambda x: TextBlob(x).sentiment.polarity
  result_df['polarity'] = result_df['reviewText'].apply(pol)

  result_sim_df = result_sim_df.merge(result_df, on='reviewText', how='inner')

  return result_sim_df

Add nearest neighbor feedback. 

In [10]:
def tokenize_and_stem(s):
    return [STEMMER.stem(t) for t 
            in TOKENIZER.tokenize(s.translate(REMOVE_PUNCTUATION_TABLE))]

In [11]:
def nn_feedback(feedback, review_list, query, result_sim_df):
  # print('*******')
  # print(result_sim_df)
  # print('*******')
  feedback_queries = list(feedback.keys())

  # feedback_query = feedback_queries[0]

  # feedback_query_tokenized = TOKENIZER.tokenize(
  #       feedback_query.translate(REMOVE_PUNCTUATION_TABLE)
  #       )

  vectorizer = TfidfVectorizer(tokenizer=tokenize_and_stem, stop_words='english')
  vectorizer.fit(review_list)

  feedback_queries = list(feedback.keys())

  similarity = cosine_similarity(vectorizer.transform([query]), 
                                vectorizer.transform(feedback_queries))
  
  #print(similarity)

  feedback_max_similarity = np.amax(similarity)

  #print(feedback_max_similarity)

  max_idx = np.argmax(similarity)

  feedback_queries[max_idx]

  pos_feedback_product_id = [idx for idx, feedback_value 
                        in feedback[feedback_queries[max_idx]] 
                        if feedback_value == 1.]

  counts = Counter(pos_feedback_product_id)

  pos_feedback_proportions = {
        doc_idx: count / sum(counts.values()) for doc_idx, count in counts.items()
  }

  pos_feedback_feature = {}
  for key in pos_feedback_proportions:
    pos_feedback_feature[key] = pos_feedback_proportions[key]*feedback_max_similarity
  
  for pid in pos_feedback_feature:
    if pid not in result_sim_df.values:
      result_sim_df = result_sim_df.append({'asin': pid, 'similarity':result_sim_df.get(pid)}, ignore_index=True)

  result_sim_df = result_sim_df[result_sim_df.groupby('asin')['similarity'].transform('max') == result_sim_df['similarity']]
  return result_sim_df

# help functions 

In [12]:
def parse_query(query):
  feature = "" 
  query_list = query.split(',')
  #print(query_list)
  count = 0
  for ele in query_list:  
    feature += ele
    feature += ' '   
  result_query_list = []
  result_query_list.append(query_list[0])
  result_query_list.append(feature)
  #print(feature)
  return result_query_list

In [13]:
def get_result(query, entire_merged_df, feedback):
  query_list = parse_query(query)
  # print(query_list[0])
  review_list_of_list = query_dataframe(query_list[0],entire_merged_df)
  # review_list_of_list
  # print(review_list_of_list)

  review_list = get_review_list(query_list[0],entire_merged_df)
  # print(query_list[1])
  sim_list = similarity(query_list[1], review_list_of_list)
  # print(sim_list)
  result_sim_df = get_result_similarity_wtfeedback(sim_list, review_list_of_list)

  result_max_sim_df = result_sim_df[result_sim_df.groupby('asin')['similarity'].transform('max') == result_sim_df['similarity']]
  result_max_sim_df = result_max_sim_df[['asin', 'reviewText','similarity']]

  #group by asin and use polarity
  grouped_result_sim_df = result_sim_df.groupby(['asin'])['polarity'].sum()
  grouped_result_sim_df = grouped_result_sim_df.reset_index()
  grouped_result_sim_df = grouped_result_sim_df[grouped_result_sim_df.polarity > 0]

  grouped_result_sim_df = grouped_result_sim_df.merge(result_max_sim_df, on='asin', how='inner')
  # grouped_result_sim_df

  result_sim_df = nn_feedback(feedback, review_list, query, grouped_result_sim_df)

  result_sim_df = result_sim_df.sort_values(['similarity', 'polarity'], ascending=[False, False])
  res_list = result_sim_df["asin"].tolist()
  return res_list[:10]
# result_sim_df

In [32]:
def update_cache(asin_rel_tuple, query, feedback):
  result_query_list = parse_query(query)
  query = result_query_list[1]
  if query not in feedback:
    feedback[query] = []
  feedback[query].append(asin_rel_tuple)
  return feedback

# Test given queries

Get queries from github:

In [81]:
!wget -nc https://raw.githubusercontent.com/Leah-Wu/SI650/main/review_data/test-queries.txt

--2020-12-12 01:29:18--  https://raw.githubusercontent.com/Leah-Wu/SI650/main/review_data/test-queries.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 601 [text/plain]
Saving to: ‘test-queries.txt’

test-queries.txt    100%[===================>]     601  --.-KB/s    in 0s      

2020-12-12 01:29:18 (32.2 MB/s) - ‘test-queries.txt’ saved [601/601]



In [83]:
query_list = []
with open('test-queries.txt') as query_file:
  for query_num, line in enumerate(query_file):
    query_list.append((query_num + 1, line))
print(query_list)

[(1, 'earphone,wireless,waterproof\n'), (2, 'earphone,noise canceling,in ear\n'), (3, 'headphone,light,long playtime\n'), (4, 'laptop,windows,game friendly,slim\n'), (5, 'laptop,student,affordable\n'), (6, 'tablet,android,slim\n'), (7, 'tablet,affordable,light-weight\n'), (8, 'case,slim,magnetic\n'), (9, 'watch,ftiness track,waterproof\n'), (10, 'watch,long battery life,bluetooth\n'), (11, 'monitor,slim,frameless,good picture quality\n'), (12, 'speaker,blutooth,long battery life\n'), (13, 'tv,smart,good picture quality\n'), (14, 'projector,portable,good picture quality\n'), (15, 'keyboard,wireless,backlight,smooth\n'), (16, 'mouse,wireless,quiet\n'), (17, 'printer,wireless,fast\n'), (18, 'mount,stable,easy install,flexible\n'), (19, 'lamp,portable,eye-caring,wireless\n'), (20, 'phone,seamless,slim')]


In [38]:
feedback = {
        'earphone wireless waterproof': [('B001GPCBPU', 1.)],
        'piano keyboard': [('B0002F52EW', 1.)]
}

In [181]:
for q_id, query in query_list:
  print(f"Searching for query: {query}")
  curr_results = get_result(query, entire_merged_df, feedback)
  for res in curr_results:
    url="https://www.amazon.com/dp/"+str(res)
    final_res.append((q_id, res, url, query))

Searching for query: earphone,wireless,waterproof

Searching for query: earphone,noise canceling,in ear

Searching for query: headphone,light,long playtime

Searching for query: laptop,windows,game friendly,slim

Searching for query: laptop,student,affordable

Searching for query: tablet,android,slim

Searching for query: tablet,affordable,light-weight

Searching for query: case,slim,magnetic

Searching for query: watch,ftiness track,waterproof

Searching for query: watch,long battery life,bluetooth

Searching for query: monitor,slim,frameless,good picture quality

Searching for query: speaker,blutooth,long battery life

Searching for query: tv,smart,good picture quality

Searching for query: projector,portable,good picture quality

Searching for query: keyboard,wireless,backlight,smooth

Searching for query: mouse,wireless,quiet

Searching for query: printer,wireless,fast

Searching for query: mount,stable,easy install,flexible

Searching for query: lamp,portable,eye-caring,wireless



In [179]:
final_res_df = pd.DataFrame(final_res, columns=["queryid","productid","product_url","query_text"])
final_res_df

,queryid,productid,product_url,query_text
0,5,B005X1Y7I2,https://www.amazon.com/dp/B005X1Y7I2,"stick,bluetooth,good remote"
1,5,B000065UDU,https://www.amazon.com/dp/B000065UDU,"stick,bluetooth,good remote"
2,5,B000JV3A20,https://www.amazon.com/dp/B000JV3A20,"stick,bluetooth,good remote"
3,5,B005GFXH6S,https://www.amazon.com/dp/B005GFXH6S,"stick,bluetooth,good remote"
4,5,B007RDWF0Q,https://www.amazon.com/dp/B007RDWF0Q,"stick,bluetooth,good remote"
...,...,...,...,...
85,19,B000I3T3K2,https://www.amazon.com/dp/B000I3T3K2,"modem,reliable,fast"
86,19,B00006HUAS,https://www.amazon.com/dp/B00006HUAS,"modem,reliable,fast"
87,19,B003Y49TO2,https://www.amazon.com/dp/B003Y49TO2,"modem,reliable,fast"
88,19,B005J8330W,https://www.amazon.com/dp/B005J8330W,"modem,reliable,fast"


In [180]:
final_res_df.to_csv("main-results.csv")

# Feedback test after annotation

After Annotation, update cache with annotation

In [15]:
!wget -nc https://github.com/Leah-Wu/SI650/raw/main/main-results-annotation.csv

--2020-12-12 17:17:38--  https://github.com/Leah-Wu/SI650/raw/main/main-results-annotation.csv
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Leah-Wu/SI650/main/main-results-annotation.csv [following]
--2020-12-12 17:17:38--  https://raw.githubusercontent.com/Leah-Wu/SI650/main/main-results-annotation.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17060 (17K) [text/plain]
Saving to: ‘main-results-annotation.csv’

main-results-annota 100%[===================>]  16.66K  --.-KB/s    in 0s      

2020-12-12 17:17:38 (158 MB/s) - ‘main-results-annotation.csv’ saved [17060/17060]



In [16]:
annotation_df = pd.read_csv("main-results-annotation.csv",encoding='utf-8')
annotation_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   queryid      200 non-null    int64 
 1   productid    200 non-null    object
 2   product_url  200 non-null    object
 3   query_text   200 non-null    object
 4   relevance    200 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 7.9+ KB


Befor feedback:

In [39]:
simlilar_query = "earphone,sweatproof,durable"
new_results = get_result(simlilar_query, entire_merged_df, feedback)
res_list=[]
for res in new_results:
  url="https://www.amazon.com/dp/"+str(res)
  res_list.append((res, url, simlilar_query))

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher',

In [41]:
feedback={}
for i in range(20):
  q_id = i+1
  curr_df = annotation_df[annotation_df['queryid']==q_id]
  query = curr_df.iloc[1]['query_text']
  pd_list = curr_df["productid"].tolist()
  rel_list = curr_df["relevance"].tolist()
  # list_of_rel = []
  for j in range(10):
    rel_tuple = (pd_list[j], rel_list[j])
    # list_of_rel.append(rel_tuple)
    feedback = update_cache(rel_tuple, query, feedback)

In [42]:
print(feedback)

{'earphone wireless waterproof\n ': [('B00275F2GS', 0), ('B0036J1S1W', 1), ('B004CHHLDG', 0), ('B001GPCBPU', 0), ('B001I1A5WS', 0), ('B001GUPEMW', 1), ('B0051RW1SQ', 0), ('B005J0RAI6', 0), ('B005J0W3F6', 0), ('B003DW4A76', 0)], 'earphone noise canceling in ear\n ': [('B001GPCBPU', 0), ('B007BWCDYQ', 0), ('B00029U0YK', 0), ('B00275F2GS', 0), ('B0036J1S1W', 0), ('B005J0W3F6', 1), ('B005J0RAI6', 1), ('B001GUPEMW', 1), ('B005IZ65G0', 1), ('B001I1A5WS', 1)], 'headphone light long playtime\n ': [('B008LSUY7W', 0), ('B00113XADE', 0), ('B00478O0JI', 1), ('B0009MFKDQ', 1), ('B002VPDOH8', 0), ('B003BRNZTM', 0), ('B0036RDVU0', 1), ('B00CXP4AS8', 1), ('B0033ZP5YA', 0), ('B0036RA4CS', 1)], 'laptop windows game friendly slim\n ': [('B004VC3NM0', 1), ('B00D4OF6SK', 1), ('B001KV0Z5I', 0), ('B008C3DEXM', 0), ('B005E99A8K', 0), ('B008F1GFWI', 0), ('B00763W9H6', 0), ('B0023XKUHA', 0), ('B009Z7NE1Q', 1), ('B002I2RHXQ', 0)], 'stick bluetooth good remote ': [('B005X1Y7I2', 0), ('B000065UDU', 1), ('B000JV3A2

Search for a similar query in cache and check for result's ndcg

In [43]:
# simlilar_query = "earphone,sweatproof,durable"
new_results = get_result(simlilar_query, entire_merged_df, feedback)
for res in new_results:
  url="https://www.amazon.com/dp/"+str(res)
  res_list.append((res, url, simlilar_query))
res_list_df = pd.DataFrame(res_list, columns=["productid","product_url","query_text"])
res_list_df.to_csv("feedback-retry.csv")

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher',